# 01 - Basic usage example
Simulation of 100 persons moving along road network obtained from OpenStreetMap (random waypoint movement)

In [ ]:
from src.IISMotion import IISMotion
from src.common.Location import Location
from src.common.CommonFunctions import CommonFunctions
from src.common.SimulationClock import *
import asyncio
import time
from src.movement.movementStrategies.MovementStrategyType import MovementStrategyType

fun = CommonFunctions()

# Setting location that will be simulated
location = Location()
location.setLatitude(48.709936)
location.setLongitude(21.238923)
location.setAltitude(0)
radius = 200  #radius around the location that will be included

oneway = False      #oneways are disabled so agents will not get stuck at the edges of simulated area
guiEnabled = True   #gui enabled, to see the agents, open /frontend/index.html in your browser while simulation running
guiTimeout = 0.5    #in seconds, sleep between simulations steps (gui is unable to keep up with updates at full speed)
gridRows = 30       #grid that world is split into (used to find closest pairs of agents when needed)

secondsPerTick = 1  #each iteration will increment clock by 1 second
noOfTicks = 500     #number of iterations simulation will take

iismotion = IISMotion(radius=radius,
                      location=location,
                      oneWayEnabled=oneway,
                      guiEnabled=guiEnabled,
                      gridRows=gridRows,
                      secondsPerTick=secondsPerTick)  # initialize IISMotion
#Adding persons with random waypoint walk along the drive network downloaded from OSM
#collection with given movement type is created
collectionOfPersons = iismotion.createActorCollection("RandomWaypointWithCuda", True, MovementStrategyType.RANDOM_WAYPOINT_CITY_CUDA)
collectionOfPersons.addPersons(100, False)      #will add 100 persons to random nodes (False means they will stay at given nodes)
collectionOfPersons.setGuiEnabled(guiEnabled)   #collaction will be visualised in GUI

#method that moves agents for desired number of iterations
#async because of "GUI"
async def simulate():
    print("=================== Simulation started ===================")
    start = time.time()
    predictions = []
    for step in range(0, noOfTicks):
        newDay = updateSimulationClock(secondsPerTick)
        print(f"---------------- step: {step} ---------------- dateTime : {getDateTime()} ----------------")
        stepStart = time.time()
        iismotion.stepAllCollections(newDay)  # move all collections with ableOfMovement=True
        stepEnd = time.time()

        print("step took ", stepEnd - stepStart)
        global DATETIME
        if (guiEnabled == True):
            await asyncio.sleep(guiTimeout)
        end = time.time()

    elapsed = end - start
    print("================== Simulation finished ===================")
    print("elapsed time:", elapsed)

#main method that will execute simulation based on guiEnabled param
def main():
    if (guiEnabled):
        loop = asyncio.get_event_loop()
        loop.create_task(simulate())
        loop.run_until_complete(iismotion.frontend.start_server)
        loop.run_forever()
    else:
        loop = asyncio.get_event_loop()
        loop.run_until_complete(simulate())

if __name__ == '__main__':
    main()